In [ ]:
import numpy as np
import os, glob, sys, pickle, itertools,warnings
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import matplotlib
from scipy.stats import *
from matplotlib.patches import Patch
import colorcet as cc
import seaborn as sns

In [ ]:
f = open('./data/TPM_maf_0to10yrs_v2.0_noDD.p','rb')
tpm = pd.DataFrame(pickle.load(f))
cum = []
med = []
for ops in tpm.index:
    m, b=np.histogram(tpm['TPM_10yrs'][ops].data[~np.isnan(tpm['TPM_10yrs'][ops].data)], bins=np.linspace(0,0.5,30))
    med.append(m)
    cum.append(np.cumsum(m))
med = np.median(med,axis=0)
med = np.append(med,[0])

ks= np.empty_like(tpm.index)
for i, ops in enumerate(tpm.index):
    m, b=np.histogram(tpm['TPM_10yrs'][ops].data[~np.isnan(tpm['TPM_10yrs'][ops].data)], bins=np.linspace(0,0.5,30))
    ks[i]=(ks_2samp(m,med).statistic)

tpm.index[np.where(ks>0.15)]

ops_fam = []
for op in tpm.index:
    o=op.split('_')
    if o[0]!= 'vary':
        ops_fam.append(o[0])
    else:
        ops_fam.append(o[0]+'_'+o[1])

ops_fam=np.unique(ops_fam)

ops_fam.sort()

In [ ]:
palette = sns.color_palette(cc.glasbey, n_colors=np.size(ops_fam))
print(palette)

In [ ]:
cm = np.concatenate([plt.get_cmap('tab20').colors,plt.get_cmap('Set3').colors])
plt.rc('xtick',labelsize=20)
plt.rc('ytick',labelsize=20)
fig, ax = plt.subplots(2,1,figsize=(15,14))
patches=[]
n=1
cmap =[]
legend_elements = []
#legend_elem = []
ax[0].step(b,med,where='post',color='k',linewidth=3,zorder=100)
#legend_elem.append(Patch(facecolor='white', edgecolor='k',label='mean', linewidth=3))
for j, (f,c) in enumerate(zip(ops_fam[::-1],palette)):
    print([f,c])
    for ops in np.sort(tpm.index):   
        if ops=='baseline_nexp2_v1.7.1_10yrs':
            continue
        if f in ops:
            a = 0.4
            cmap.append(matplotlib.colors.rgb2hex(c))
            nbin= 30 #int(0.5*(len(tpm[ops]['TPM'].data[~np.isnan(tpm[ops]['TPM'].data)])**(1/3))/(3.49*np.std(tpm[ops]['TPM'].data[~np.isnan(tpm[ops]['TPM'].data)])))
            ax[1].hist(tpm['TPM_10yrs'][ops].data[~np.isnan(tpm['TPM_10yrs'][ops].data)], bins=np.linspace(0,0.5,nbin), cumulative=True,density=True, histtype='step', alpha=a,color=c,lw=3)
            ax[0].hist(tpm['TPM_10yrs'][ops].data[~np.isnan(tpm['TPM_10yrs'][ops].data)], bins=np.linspace(0,0.5,nbin), cumulative=False,density=False, histtype='step', alpha=a,color=c,lw=3)
            ax[0].set_xticklabels([])
            ax[1].set_xlabel('transients recovery rate', fontsize=32)
            ax[1].set_ylabel('cumulative fraction', fontsize=32)
            ax[0].set_ylabel(' N', fontsize=32)
            
    legend_elements.append(Patch(facecolor='white', edgecolor=c,label=f, linewidth=3, alpha =a))
legend_elements.append(Patch(facecolor='white', edgecolor='k',label='mean', linewidth=3))
#ax[0].legend(handles=legend_elem,bbox_to_anchor=(0,0.8), loc='lower left',fontsize=20)                   
ax[1].legend(handles=legend_elements,ncol=3,bbox_to_anchor=(0.,1.45), loc='lower left',fontsize=18)
plt.subplots_adjust(hspace=0.05)   #legl = leg.get_lines()(-0.02, -2.92)
ax[1].set_xlim([-0.001,0.505])
ax[0].set_xlim([-0.001,0.505])
ax[1].set_ylim([0,1.01])
plt.savefig('TPM_10.pdf')